# Decision Tree (ID3)

In [1]:
import numpy as np
import pandas as pd
# Very small number to avoid log(0)
eps = np.finfo(float).eps

In [2]:
# Create dataset
dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}

In [3]:
# Dataset do DataFrame
df = pd.DataFrame(dataset,columns=['Taste','Temperature','Texture','Eat'])

In [4]:
# Show dataset
df

,Taste,Temperature,Texture,Eat
0,Salty,Hot,Soft,No
1,Spicy,Hot,Soft,No
2,Spicy,Hot,Hard,Yes
3,Spicy,Cold,Hard,No
4,Spicy,Hot,Hard,Yes
5,Sweet,Cold,Soft,Yes
6,Salty,Cold,Soft,No
7,Sweet,Hot,Soft,Yes
8,Spicy,Cold,Soft,Yes
9,Salty,Hot,Hard,Yes


In [5]:
# Column with classes
label_col = 'Eat'

<img src="./Images/entropy.jpeg">

Source = <a href="https://en.wikipedia.org/wiki/ID3_algorithm"> Wikipedia </a>

In [6]:
# Entropy for all data
def entropy_all_data(data, label_col):
    entropy = 0
    # Uniques values in label column (Yes/ No)
    uniques = data[label_col].unique()
    # For each unique value
    for unique in uniques:
        # Number of each uniques values in label column / len all data
        fraction = data[label_col].value_counts()[unique] / len(data)
        # Entropy formula
        entropy += -fraction * np.log2(fraction)
    return entropy

<img src="./Images/gain.jpeg">

Source = <a href="https://en.wikipedia.org/wiki/ID3_algorithm"> Wikipedia </a>

In [7]:
# Gain function
def gain_function(data, label_col, H):
    # Len all data
    num_all_data = len(data)
    # Targets ('Yes' / 'No')
    targets = df[label_col].unique()
    # Entropy for each column
    E_average_column = []
    # Each columns except label column
    for column in data.columns[:-1]:
        # Unique values in column
        uniques = data[column].unique()
        # Entropy for each unique value in column
        E_average_unique = []
        # Each unique values
        for unique in uniques:
            # Entropy 0
            entropy = 0
            # Len of data in column
            num_all_column = len(data[(data[column] == unique)])
            # For each target (Yes / No)
            for target in targets:
                # Number of unique values in column ('Yes' / 'No')
                num_yes_no = len(data[(data[column] == unique) & (data[label_col] == target)])
                # Number of each uniques values in column / len of all data in this column
                fraction = num_yes_no/num_all_column
                # Add to entropy
                entropy += -fraction * np.log2(eps+fraction)
            # Entropy for each unique value
            E_average_unique.append((num_all_column/num_all_data)*entropy)
        # Gain for each column
        E_average_column.append(H - np.sum(E_average_unique))
    return E_average_column

In [8]:
# Build tree
def build_tree(data, tree_dict=None):
    # Entropy for all data
    H = entropy_all_data(data, label_col)
    # Gain
    gain = gain_function(data, label_col, H)
    # Winner column
    winner_col = data.columns[np.argmax(gain)]
    # If tree_dict is None, create tree
    if tree_dict is None:                    
        tree_dict={}
        tree_dict[winner_col] = {}
    # Each unique value in winner column
    for value in np.unique(data[winner_col]):
        # Part of tree with winner column
        grab_part = data[data[winner_col] == value]
        # Unique values (Yes / No) in part of tree
        unique_labels = np.unique(grab_part[label_col])
        # If only the values of one class were left
        if len(unique_labels) == 1:
            # Add class to tree_dict
            tree_dict[winner_col][value] = unique_labels[0]
        # Else recursively call functions
        else:
            # Add result of recursively call functions to tree_dict
            tree_dict[winner_col][value] = build_tree(grab_part)
    return tree_dict

In [9]:
# Build tree
tree = build_tree(df)

In [10]:
# Library to preety print
import pprint
# Print tree
pprint.pprint(tree)

{'Taste': {'Salty': {'Texture': {'Hard': 'Yes', 'Soft': 'No'}},
           'Spicy': {'Temperature': {'Cold': {'Texture': {'Hard': 'No',
                                                          'Soft': 'Yes'}},
                                     'Hot': {'Texture': {'Hard': 'Yes',
                                                         'Soft': 'No'}}}},
           'Sweet': 'Yes'}}


In [11]:
# Prediction function
def predict(inst, tree):
    # For every nodes
    for nodes in tree.keys():
        # Grab selected columns from sample
        value = inst[nodes]
        # Add selected column to dict
        tree = tree[nodes][value]
        # If tree is dict recursively call functions
        if type(tree) is dict:
            prediction = predict(inst, tree)
        # Else final prediction
        else:
            prediction = tree
            break
    return prediction

In [12]:
# Test example
inst = df.iloc[6]
inst

Taste          Salty
Temperature     Cold
Texture         Soft
Eat               No
Name: 6, dtype: object

In [13]:
# Prediction
predict(inst, tree)

'No'

# Decision Tree (CART)

In [14]:
# Gini function
def gini(data, label_col):
    # List to grab gini values
    gini_tab = []
    # Len of all data
    num_all = len(data)
    # Targets ('Yes' / 'No')
    targets = df[label_col].unique()
    # Each column except label column
    for column in data.columns[:-1]:
        # Gini for each class
        gini_class = []
        # Every unique value in every column
        for unique in data[column].unique():
            # Gini
            g = 1
            # Num of each unique values in class
            num_all_unique = len(data[(data[column] == unique)])
            # For each target (Yes / No)
            for target in targets:
                # Number of unique values in column ('Yes' / 'No')    
                num_yes_no = len(data[(data[column] == unique) & (data[label_col] == target)])
                # Gini for each target
                g -= (num_yes_no/num_all_unique)**2
            # Gini for each unique value
            gini_class.append((num_all_unique/num_all)*g)
        # Gini for each class
        gini_tab.append(np.sum(gini_class))
    return gini_tab

In [15]:
# Build tree
def build_tree(data, tree_dict=None):
    # Gini
    gini_value = gini(data, label_col)
    # Winner column
    winner_col = data.columns[np.argmin(gini_value)]
    # If tree_dict is None, create tree
    if tree_dict is None:                    
        tree_dict={}
        tree_dict[winner_col] = {}
    # Each unique value in winner column
    for value in np.unique(data[winner_col]):
        # Part of tree with winner column
        grab_part = data[data[winner_col] == value]
        # Unique values (Yes / No) in part of tree
        unique_labels = np.unique(grab_part[label_col])
        # If only the values of one class were left
        if len(unique_labels) == 1:
            # Add class to tree_dict
            tree_dict[winner_col][value] = unique_labels[0]
        # Else recursively call functions
        else:
            # Add result of recursively call functions to tree_dict
            tree_dict[winner_col][value] = build_tree(grab_part)
    return tree_dict

In [16]:
tree_gini = build_tree(df)

In [17]:
import pprint
pprint.pprint(tree_gini)

{'Taste': {'Salty': {'Texture': {'Hard': 'Yes', 'Soft': 'No'}},
           'Spicy': {'Temperature': {'Cold': {'Texture': {'Hard': 'No',
                                                          'Soft': 'Yes'}},
                                     'Hot': {'Texture': {'Hard': 'Yes',
                                                         'Soft': 'No'}}}},
           'Sweet': 'Yes'}}


In [18]:
# Prediction function
def predict(inst, tree):
    # For every nodes
    for nodes in tree.keys():
        # Grab selected columns from sample
        value = inst[nodes]
        # Add selected column to dict
        tree = tree[nodes][value]
        # If tree is dict recursively call functions
        if type(tree) is dict:
            prediction = predict(inst, tree)
        # Else final prediction
        else:
            prediction = tree
            break
    return prediction

In [19]:
# Test example
inst = df.iloc[6]
inst

Taste          Salty
Temperature     Cold
Texture         Soft
Eat               No
Name: 6, dtype: object

In [20]:
# Prediction
predict(inst, tree_gini)

'No'